In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import dm4bem

### Introduction

In this first part we will define our working environment. The plan of our house can be found below. The objective of this study case is the nalysis of a cubic building with 5 identical walls & a transparent wall (glass window), air infiltration, and HVAC system controlling the indoor air temperature.

### Design model

The room that we will study is presented in the figure below. Two rooms are separated by a small wall without insulation. The total area of the model is about 9 meters square and the ceiling is 3 meters high. The rooms have each 3 exterior walls and one interior wall. We will consider the house as insulated from the outside .
The interior walls are made of concrete. The room also has a 1 m² window, a 2 m² door and a HVAC device.



<center><img src="https://hub.gke2.mybinder.org/user/cghiaus-dm4bem-19y1e1gf/files/Sch%C3%A9ma%20maison.png?_xsrf=2%7C3ce45c22%7Cea41507d6477cef712308b01cb5ff2db%7C1682345682"></center>

> Figure 1. Plan of the building



We will now define boundary conditions wich are important to solve our case.


### Hypothesis

Boundary Conditions:

- Temperature of the exterior air = 0 °C
- Setpoint temperature of the interior air = 18 °C
- Unique temperature on each wall
- Unidirectionnal heat transfer

### Dimension of windows, doors, and walls.

In [10]:
l = 3               # m length of the cubic room
Sg = l**2           # m² surface of the glass wall
Sc = Si = 5 * Sg    # m² surface of concrete & insulation of the 5 walls

Thermo-physical properties
The thermophysical properties of the air (in SI units) are:

In [11]:
air = {'Density': 1.2,                      # kg/m³
       'Specific heat': 1000}               # J/(kg·K)
# pd.DataFrame.from_dict(air, orient='index', columns=['air'])
pd.DataFrame(air, index=['Air'])

,Density,Specific heat
Air,1.2,1000


We use concrete, glass and insulation (glass wool) as materials. We take our door with oak wood. 

In [14]:
BETON = {'Conductivity': 1.400,
            'Density': 2400.0,
            'Specific heat': 880,
            'Width': 0.2,
            'Surface': 60}

ISOLANT = {'Conductivity': 0.046,
              'Density': 25.0,
              'Specific heat': 1030,
              'Width': 0.08,
              'Surface':45}

VERRE = {'Conductivity': 1.4,
         'Density': 2500,
         'Specific heat': 1210,
         'Width': 0.04,
         'Surface': 40}

PORTE = {'Conductivity': 0.15,
         'Density': 800,
         'Specific heat': 2000,
         'Width': 0.1,
         'Surface': 4}

wall = pd.DataFrame.from_dict({'Layer_out': ISOLANT,
                               'Layer_in': BETON,
                               'Glass': VERRE,
                               'Door': PORTE},
                              orient='index')
wall

,Conductivity,Density,Specific heat,Width,Surface
Layer_out,0.046,25.0,1030,0.08,45
Layer_in,1.400,2400.0,880,0.20,60
Glass,1.400,2500.0,1210,0.04,40
Door,0.150,800.0,2000,0.10,4


### Thermal circuit

We will now able to draw our thermal circuit. The objective with this one will be to computerize our model. The figure will be found below.

<center><img src="https://hub.gke2.mybinder.org/user/cghiaus-dm4bem-19y1e1gf/files/Sch%C3%A9ma%20%C3%A9lec.png?_xsrf=2%7C3ce45c22%7Cea41507d6477cef712308b01cb5ff2db%7C1682345682"></center>

>Figure 2. Thermal circuit


In [ ]:
#Initial Values#

Text = 0                        #°C
Tvent = ???                     #°C
Tint1 = 20                      #°C
Tint2 = 20                      #°C
lambda_concrete = 1.4
lambda_insulation = 0.046
lambda_glass = 1.4
lambda_wood = 0.15
hi = 8
ho = 25


Swall = 9
Swindow = 1
Sdoor = 2
Sint = Swall*8 - Swindow - Sdoor


# Vector of temperature sources
# =============================


f = np.zeros(20)
f[0] = 1
f[4] = 1
f[6] = 1
f[7] = 1
f[9] = 1
f[13] = 1


b = np.zeros(26)
b[0] = 1
b[8] = 1
b[10] = 1
b[11] = 1
b[18] = 1
b[20] = 1
b[21] = 1

[As, Bs, Cs, Ds] = dm4bem.tc2ss(A, G, b, C, f, y)
print('As = \n', As, '\n')
print('Bs = \n', Bs, '\n')
print('Cs = \n', Cs, '\n')
print('Ds = \n', Ds, '\n')




ytc = np.linalg.inv(A2 @ G @ A) @ (A2 @ G @ b + f)
y = np.zeros(20)
y[19]=1
u = np.hstack([b[np.nonzero(b)], f[np.nonzero(f)]])

[As, Bs, Cs, Ds] = dm4bem.tc2ss(A, G, b, C, f, y)
yss = (-Cs @ np.linalg.inv(As) @ Bs + Ds) @ u


u = np.hstack([b[np.nonzero(b)], f[np.nonzero(f)]])

# Maximum time-step
dtmax = min(-2. / np.linalg.eig(As)[0])
print(f'Maximum time step: {dtmax:.2f} s')
dt = 360
print(f'Time step: {dt:.2f} s')



# Step response
# -------------
duration = 3600 * 24 * 10        # [s]
# number of steps
n = int(np.floor(duration / dt))

t = np.arange(0, n * dt, dt)    # time

# Vectors of state and input (in time)
n_tC = As.shape[0]              # no of state variables (temps with capacity)

# u = [To To To Tsp Phio Phii Qaux Phia]
u = np.zeros([18, n])
u[0:7, :] = np.ones([7, n])






